In [38]:
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
from tqdm import tqdm
from glob import glob

display(HTML("<style>.container { width:100% !important; }</style>"))

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

In [2]:
def convert_season_to_int(season):
    season = season.split('-')[0]
    return int(season) + 1

def convert_int_to_season(year):
    previuos_year = year - 1
    year = str(year)
    previous_year = str(previuos_year)
    year = year[-2:]
    return f'{previous_year}-{year}'

In [3]:
def getSoupFromURL(url):

    try:
        r = requests.get(url)
    except:
        return None

    return BeautifulSoup(r.text, "html.parser")

def relative_table(table, avg_table, skip_names):
    for column_name in table.columns:
        if column_name in skip_names:
            continue
    
        table[f'{column_name}'] = (table[f'{column_name}'] / avg_table[f'{column_name}']) - 1
        
    return table

def get_bbref_teams_dataframe(first_year,
                              last_year,
                              find_champion=True,
                              save=True,
                              relative=True,
                              time_to_sleep=2):

    all_teams = pd.DataFrame()

    for i, year in tqdm(enumerate(range(first_year, last_year + 1)), total=last_year + 1 - first_year):

        url = f'https://www.basketball-reference.com/leagues/NBA_{year}.html'

        advanced_table_id = "advanced-team"
        per_game_table_id = 'per_game-team'

        soup = getSoupFromURL(url)
        time.sleep(time_to_sleep)

        advanced_table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']==advanced_table_id)
        per_game_table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']==per_game_table_id)
        
        if find_champion:
            champion = soup.find(lambda tag: tag.name=='div' and tag.has_attr('id') and tag['id']=='info')
            champion_name = " ".join(champion.find_all('div')[2].find('p').text.split(" ")[2:])

        advanced_stats = pd.read_html(str(advanced_table), header=1)[0]
        advanced_stats = advanced_stats.drop(['Unnamed: 17', 'Unnamed: 22', 'Unnamed: 27', 'Arena', 'Attend.'], axis=1)
        advanced_league_avg = advanced_stats.iloc[-1]
        advanced_stats = advanced_stats[:-1]

        per_game_stats = pd.read_html(str(per_game_table), header=0)[0]
        per_game_league_avg = per_game_stats.iloc[-1]
        per_game_stats = per_game_stats[:-1]
        
        if relative:
            per_game_stats = relative_table(per_game_stats, per_game_league_avg, skip_names=['Team', 'Rk'])
            advanced_stats = relative_table(advanced_stats, advanced_league_avg, skip_names=['Team', 'Rk', 'W', 'L', 'MOV', 'SOS', 'SRS', 'NRtg'])

        combined_team_stats = pd.merge(per_game_stats, advanced_stats, on=['Team'])
        combined_team_stats['Team'] = combined_team_stats['Team'].str.replace('*', '')

        combined_team_stats['Season'] = convert_int_to_season(year)
        combined_team_stats['Win%'] = combined_team_stats['W'] / (combined_team_stats['W'] + combined_team_stats['L'])
        
        combined_team_stats = combined_team_stats.fillna(0)
        combined_team_stats = combined_team_stats.drop(['Rk_x', 'Rk_y', 'G', 'MP'], axis=1)
        
        if find_champion:
            combined_team_stats['Champion'] = 0
            combined_team_stats.loc[combined_team_stats['Team'] == champion_name, 'Champion'] = 1
            
            if combined_team_stats.Champion.sum() != 1:
                print(f'[ERROR] - {year}')
                break
                
        all_teams = pd.concat([all_teams, combined_team_stats])
        
        time.sleep(5)

    if save:
        all_teams.to_csv('historical_relative_data.csv')
    
    return all_teams

In [4]:
first_year = 1980
last_year = 2024

file_name = 'historical_relative_data.csv'
if os.path.exists(file_name):
    print('File exists - loading from disk')
    teams = pd.read_csv(file_name)
else:
    teams = get_bbref_teams_dataframe(first_year, last_year, save=True)

File exists - loading from disk


In [5]:
teams

Unnamed: 0                    Team        FG       FGA       FG%  \
0              0       San Antonio Spurs  0.077982  0.041943  0.035343   
1              1      Los Angeles Lakers  0.089450 -0.007726  0.099792   
2              2     Cleveland Cavaliers  0.066514  0.082781 -0.014553   
3              3         New York Knicks  0.064220  0.033113  0.031185   
4              4          Boston Celtics  0.011468 -0.005519  0.018711   
...          ...                     ...       ...       ...       ...   
1249          25              Miami Heat -0.056872 -0.037120 -0.018987   
1250          26         Detroit Pistons -0.030806 -0.007874 -0.023207   
1251          27       Charlotte Hornets -0.052133 -0.021372 -0.029536   
1252          28  Portland Trail Blazers -0.066351  0.008999 -0.073840   
1253          29       Memphis Grizzlies -0.090047 -0.007874 -0.082278   

            3P       3PA       3P%        2P       2PA  ...      ORB%  \
0    -0.250000 -0.107143 -0.100000  0.081585  0.045506  ... -0.050746   
1    -0.750000 -0.571429 -0.285714  0.102564  0.007964  ... -0.026866   
2    -0.500000 -0.178571 -0.310714  0.072261  0.089875  ... -0.011940   
3    -0.375000 -0.178571 -0.214286  0.069930  0.037543  ...  0.005970   
4     1.500000  0.821429  0.371429 -0.018648 -0.034130  ...  0.038806   
...        ...       ...       ...       ...       ...  ...       ...   
1249 -0.023438 -0.039886  0.010929 -0.064846 -0.035316  ... -0.099174   
1250 -0.140625 -0.096866 -0.049180  0.017065  0.050186  ... -0.012397   
1251 -0.054688 -0.031339 -0.030055 -0.044369 -0.014870  ... -0.128099   
1252 -0.101562 -0.054131 -0.057377 -0.047782  0.050186  ...  0.136364   
1253  0.023438  0.076923 -0.054645 -0.136519 -0.063197  ... -0.016529   

        FT/FGA    eFG%.1    TOV%.1      DRB%  FT/FGA.1  Attend./G   Season  \
0     0.114894  0.041152 -0.070968  0.004511 -0.080851  -0.345574  1979-80   
1    -0.063830 -0.022634 -0.096774  0.006015 -0.229787  -0.720000  1979-80   
2    -0.097872  0.039095  0.051613 -0.009023 -0.080851  -0.220328  1979-80   
3    -0.059574  0.024691  0.058065 -0.037594  0.119149  -0.060984  1979-80   
4     0.097872 -0.022634  0.064516  0.019549 -0.004255   8.501639  1979-80   
...        ...       ...       ...       ...       ...        ...      ...   
1249  0.093750 -0.007313  0.049587  0.030343 -0.093750   0.077469  2023-24   
1250  0.005208  0.020110 -0.082645  0.019789  0.151042  -0.009329  2023-24   
1251 -0.130208  0.045704  0.016529 -0.017150 -0.020833  -0.102619  2023-24   
1252 -0.057292  0.020110  0.057851 -0.023747  0.125000  -0.000164  2023-24   
1253 -0.052083  0.007313  0.115702 -0.013193  0.015625  -0.092690  2023-24   

          Win%  Champion  
0     0.500000         0  
1     0.731707         1  
2     0.451220         0  
3     0.475610         0  
4     0.743902         0  
...        ...       ...  
1249  0.560976         0  
1250  0.170732         0  
1251  0.256098         0  
1252  0.256098         0  
1253  0.329268         0  

[1254 rows x 50 columns]

In [11]:
corr = teams.select_dtypes(include=np.number).corr()
corr.style.background_gradient(cmap='coolwarm').format(precision=2)

In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from catboost import CatBoostClassifier, Pool


def create_and_train_svm(x_train, y_train, x_test):
    clf = make_pipeline(SVC(gamma='auto', probability=True))
    clf.fit(x_train, y_train)
    
    probs = clf.predict_proba(x_test)
    return 100 * probs[:,1]

def create_and_train_xgb(x_train, y_train, x_test):

    dtrain = xgb.DMatrix(x_train, label=y_train)
    param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    
    evallist = [(dtrain, 'train')]
    
    num_round = 100
    bst = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)
    dtest = xgb.DMatrix(x_test)
    ypred = bst.predict(dtest)
    return 100 * ypred

def create_and_train_lr(x_train, y_train, x_test):
    lr = LogisticRegression(
        penalty='l2',
        solver='newton-cholesky',
        C=0.1
    )
    lr.fit(x_train, y_train)
    probs = lr.predict_proba(x_test)[:,1]
    
    lr_probs = 100 * probs
    return lr_probs

def create_and_train_lda(x_train, y_train, x_test):
    clf = LinearDiscriminantAnalysis()
    clf.fit(x_train, y_train)
    
    lda_probs = clf.predict_proba(x_test)
    return 100 * lda_probs[:,1]

def create_and_train_cat_boost(x_train, y_train, x_test):
    model = CatBoostClassifier(iterations=20,
                               depth=2,
                               learning_rate=0.1,
                               loss_function='Logloss',
                               verbose=False)
    model.fit(x_train, y_train)
    preds_proba = model.predict_proba(x_test)
    return 100 * preds_proba[:,1]
    

def select_features(x_train, y_train, x_test, k=10):
    selector = SelectKBest(score_func=f_classif, k=k)
    x_train_selected = selector.fit_transform(x_train, y_train)
    x_test_selected = selector.transform(x_test)
    return x_train_selected, x_test_selected, selector

def create_and_train_big_model(x_train, y_train, x_test, k=10):
    x_train_selected, x_test_selected, selector = select_features(x_train, y_train, x_test, k)

    model_probs = []

    # model_probs.append(create_and_train_svm(x_train_selected, y_train, x_test_selected))
    model_probs.append(create_and_train_xgb(x_train_selected, y_train, x_test_selected))
    # model_probs.append(create_and_train_lr(x_train_selected, y_train, x_test_selected))
    # model_probs.append(create_and_train_lda(x_train_selected, y_train, x_test_selected))
    model_probs.append(create_and_train_cat_boost(x_train, y_train, x_test))

    # Step 3: Average probabilities
    # If no models are selected, we return an empty array or handle the case where no models are used
    if model_probs:
        avg_probs = sum(model_probs) / len(model_probs)
        return avg_probs
    else:
        # Handle the case where no models are selected (optional)
        raise ValueError("No models selected. Please uncomment at least one model.")

In [102]:
teams['Season'] = teams['Season'].astype(str)

# Get unique seasons
seasons = teams['Season'].unique()
print_results = True
save = False

# K-folds by leave-one-season-out
results = []  # to store all season-wise records

results = []  # to store all season-wise records
correct_predictions = 0  # to accumulate the number of correct predictions

for i, season in enumerate(seasons):
    test_df = teams[teams['Season'] == season].reset_index(drop=True)
    train_df = teams[teams['Season'] != season].reset_index(drop=True)

    x_train = train_df.drop(columns=["Champion", "Season", "Team"])
    y_train = train_df[["Champion"]]

    x_test = test_df.drop(columns=["Champion", "Season", "Team"])
    y_test = test_df[["Champion"]]

    test_df["avg_prob"] = create_and_train_big_model(x_train, y_train, x_test, k=15)

    # Predicted champion
    predicted_row = test_df.loc[test_df["avg_prob"].idxmax()]
    predicted_team = predicted_row["Team"]
    predicted_prob = predicted_row["avg_prob"]

    # Actual champion
    true_champion_row = test_df[test_df["Champion"] == 1]
    if not true_champion_row.empty:
        true_team = true_champion_row.iloc[0]["Team"]
        true_prob = true_champion_row.iloc[0]["avg_prob"]
    else:
        true_team = None
        true_prob = None

    correct = predicted_team == true_team
    correct_predictions += correct  # Increment correct prediction counter

    # Store result
    results.append({
        "Season": season,
        "Predicted Champion": predicted_team,
        "Predicted Prob": round(predicted_prob, 2),
        "Actual Champion": true_team,
        "Actual Prob": round(true_prob, 2) if true_prob is not None else None,
        "Correct": correct
    })

    if print_results:
        # Optional console output
        print(f"\n📅 Season: {season}")
        print(f"🏆 Predicted Champion: {predicted_team} (Prob: {predicted_prob:.2f}%)")
        print(f"👑 Actual Champion:    {true_team} (Prob: {true_prob:.2f}%)")
        print("✅ Correct prediction!" if correct else "❌ Wrong prediction!")

# Calculate accuracy after all seasons have been processed
total_seasons = len(seasons)
accuracy = (correct_predictions / total_seasons) * 100

print(f"\n🎯 Overall Prediction Accuracy: {accuracy:.2f}% ({correct_predictions} out of {total_seasons} seasons)")

if save:
    # Optionally save the results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv("champion_predictions.csv", index=False)
    print(f"\n📁 Results saved to 'champion_predictions.csv'")



📅 Season: 1979-80
🏆 Predicted Champion: Los Angeles Lakers (Prob: 18.45%)
👑 Actual Champion:    Los Angeles Lakers (Prob: 18.45%)
✅ Correct prediction!

📅 Season: 1980-81
🏆 Predicted Champion: Boston Celtics (Prob: 41.23%)
👑 Actual Champion:    Boston Celtics (Prob: 41.23%)
✅ Correct prediction!

📅 Season: 1981-82
🏆 Predicted Champion: Boston Celtics (Prob: 51.69%)
👑 Actual Champion:    Los Angeles Lakers (Prob: 11.59%)
❌ Wrong prediction!

📅 Season: 1982-83
🏆 Predicted Champion: Philadelphia 76ers (Prob: 12.78%)
👑 Actual Champion:    Philadelphia 76ers (Prob: 12.78%)
✅ Correct prediction!

📅 Season: 1983-84
🏆 Predicted Champion: Boston Celtics (Prob: 13.70%)
👑 Actual Champion:    Boston Celtics (Prob: 13.70%)
✅ Correct prediction!

📅 Season: 1984-85
🏆 Predicted Champion: Los Angeles Lakers (Prob: 18.87%)
👑 Actual Champion:    Los Angeles Lakers (Prob: 18.87%)
✅ Correct prediction!

📅 Season: 1985-86
🏆 Predicted Champion: Boston Celtics (Prob: 67.06%)
👑 Actual Champion:    Boston Celt

In [77]:
champions_files = glob('champions_predictions/*47*')

if len(champions_files) > 0:
    for file in champions_files:
        results_df = pd.read_csv(file)
        
        # Calculate accuracy
        accuracy = results_df["Correct"].mean() * 100
        
        # Display accuracy for the current k value
        print(f"✅ Prediction Accuracy for K ({file}): {accuracy:.2f}% ({results_df['Correct'].sum()} out of {len(results_df)} seasons)")
else:
    print('No predictions were made')
    

✅ Prediction Accuracy for K (champions_predictions/champion_predictions_47.csv): 35.56% (16 out of 45 seasons)


In [99]:
def load_current_year():
    current_year = get_bbref_teams_dataframe(first_year=2025,
                                             last_year=2025,
                                             find_champion=False,
                                             save=False)
    
    current_year_with_metrics = pd.DataFrame(current_year)
    return current_year_with_metrics

def predict_champions(teams, current_year):
    # Set the test set to the current year and train on all available historical data
    test_df = current_year.reset_index(drop=True)
    train_df = teams.reset_index(drop=True)

    # Separate features and labels for training
    x_train = train_df.drop(columns=["Champion", "Season", "Team"])
    print(x_train)

    y_train = train_df[["Champion"]]

    # Prepare features for testing (current year data)
    x_test = test_df.drop(columns=["Season", "Team"])
    print(x_test)

    # Make predictions using the trained model
    test_df["avg_prob"] = create_and_train_big_model(x_train, y_train, x_test, k=15)

    # Collect predicted teams and their probabilities
    predictions = test_df[["Team", "avg_prob"]].sort_values(by="avg_prob", ascending=False)

    # Display the DataFrame of predictions
    print("Predicted Teams and their Championship Probabilities:")
    print(predictions)

    # Output the predicted champion (team with the highest probability)
    predicted_row = predictions.iloc[0]
    predicted_team = predicted_row["Team"]
    predicted_prob = predicted_row["avg_prob"]
    print(f"\n🏆 Predicted Champion for 2025: {predicted_team} (Prob: {predicted_prob:.2f}%)")
    
    return predictions, predicted_team, predicted_prob

In [95]:
current_year_data = load_current_year()


100%|██████████| 1/1 [00:07<00:00,  7.66s/it]


In [100]:

predictions, predicted_team, predicted_prob = predict_champions(teams, current_year_data)

      Unnamed: 0        FG       FGA       FG%        3P       3PA       3P%  \
0              0  0.077982  0.041943  0.035343 -0.250000 -0.107143 -0.100000   
1              1  0.089450 -0.007726  0.099792 -0.750000 -0.571429 -0.285714   
2              2  0.066514  0.082781 -0.014553 -0.500000 -0.178571 -0.310714   
3              3  0.064220  0.033113  0.031185 -0.375000 -0.178571 -0.214286   
4              4  0.011468 -0.005519  0.018711  1.500000  0.821429  0.371429   
...          ...       ...       ...       ...       ...       ...       ...   
1249          25 -0.056872 -0.037120 -0.018987 -0.023438 -0.039886  0.010929   
1250          26 -0.030806 -0.007874 -0.023207 -0.140625 -0.096866 -0.049180   
1251          27 -0.052133 -0.021372 -0.029536 -0.054688 -0.031339 -0.030055   
1252          28 -0.066351  0.008999 -0.073840 -0.101562 -0.054131 -0.057377   
1253          29 -0.090047 -0.007874 -0.082278  0.023438  0.076923 -0.054645   

            2P       2PA       2P%  ...

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Unnamed: 0


In [28]:
champion_columns = current_year_with_metrics.filter(like='_Champion%')
average_champion = current_year_with_metrics.filter(like='_Champion%').mean(axis=1)
current_year_with_metrics['AVG_Champion%'] = average_champion
columns_to_display = ['Team']
columns_to_display.extend(list(champion_columns.columns))

current_year_with_metrics.sort_values(by=['AVG_Champion%'], ascending=False)[columns_to_display]

Team  SVM_Champion%  XGBoost_Champion%  LDA_Champion%  \
3    Oklahoma City Thunder      73.948009           1.103692      27.031195   
7           Boston Celtics       2.974627          69.340691      22.307777   
0      Cleveland Cavaliers      31.577536           0.494801      26.389707   
13         Houston Rockets       2.586880           0.008484       0.131137   
8          New York Knicks       2.695908           0.000578       0.475712   
2           Denver Nuggets       2.783013           0.003334       0.551270   
18      Los Angeles Lakers       2.736891           0.004349       0.600456   
6           Indiana Pacers       2.715239           0.000258       0.414123   
19    Los Angeles Clippers       2.816171           0.104569       0.231438   
1        Memphis Grizzlies       2.821283           0.000069       0.280122   
16   Golden State Warriors       2.790173           0.001505       0.340306   
15  Minnesota Timberwolves       2.820044           0.000027       0.234051   
11         Milwaukee Bucks       2.780701           0.000232       0.360505   
10         Detroit Pistons       2.735012           0.001605       0.275530   
12        Dallas Mavericks       2.798889           0.000358       0.416717   
14       San Antonio Spurs       2.810164           0.000115       0.455905   
4            Atlanta Hawks       2.817844           0.000584       0.315158   
26      Washington Wizards       3.108851           0.000247       0.000582   
21  Portland Trail Blazers       2.827488           0.000251       0.173537   
28           Orlando Magic       2.812162           0.000056       0.102188   
5            Chicago Bulls       2.816420           0.000011       0.166561   
17            Phoenix Suns       2.822085           0.000195       0.138278   
9         Sacramento Kings       2.770878           0.001373       0.030419   
22         Toronto Raptors       2.864566           0.001922       0.038628   
20               Utah Jazz       2.894963           0.000046       0.002191   
23              Miami Heat       2.819868           0.000300       0.015313   
27       Charlotte Hornets       2.852662           0.000213       0.004665   
24    New Orleans Pelicans       2.836060           0.000043       0.016785   
29           Brooklyn Nets       2.844168           0.000174       0.005601   
25      Philadelphia 76ers       2.822519           0.000046       0.010118   

    LR_Champion%  AVG_Champion%  
3      71.340033      43.355732  
7      29.489392      31.028122  
0      53.773139      28.058796  
13      5.235132       1.990408  
8       2.945855       1.529513  
2       1.340768       1.169596  
18      1.285345       1.156760  
6       1.488419       1.154510  
19      1.371250       1.130857  
1       1.368765       1.117560  
16      1.199290       1.082818  
15      1.029329       1.020863  
11      0.661797       0.950809  
10      0.405357       0.854376  
12      0.093286       0.827312  
14      0.018360       0.821136  
4       0.064122       0.799427  
26      0.000185       0.777466  
21      0.036372       0.759412  
28      0.119032       0.758360  
5       0.050064       0.758264  
17      0.035687       0.749061  
9       0.132697       0.733842  
22      0.007273       0.728097  
20      0.000176       0.724344  
23      0.060584       0.724016  
27      0.000419       0.714490  
24      0.000707       0.713399  
29      0.002669       0.713153  
25      0.001279       0.708490

['SVM_Champion%',
 'XGBoost_Champion%',
 'LDA_Champion%',
 'LR_Champion%',
 'AVG_Champion%']